In [64]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# set pandas configuration to show all columns
pd.set_option('display.max_columns', None)

In [65]:
def find_fill_missing_dates(df, aggregator, date_col, fill_cols, copy_val_cols= None, start= None, end= None, fill_value= np.nan, fill_val_dict= None):
    '''
    Identifies missing dates, fills missing dates with NaN
    ----
    inputs 
    df: Pandas DataFrame, 
    aggregator: aggregation column (string)
    date_col: primary date-time column (string)
    fill_cols: columns to include (list)
    start (optional): start-date for zero-filling (date-time object)
    end (optional): end-date for zero-filling (date-time object)
    fill_value: default value to place in missing rows + cols (string, numeric, bool)
    fill_val_dict: dictionary of column:fill_value for missing rows (dictionary)

    ----
    outputs: Pandas DataFrame
    '''
    new_df = pd.DataFrame()
    for item in tqdm(df[aggregator].unique().tolist()):
        data = df[df[aggregator] == item]
        s = data[date_col].min()
        e = data[date_col].max()
        if start is not None:
            s = start
        if end is not None:
            e = end
        missing_dates = pd.DataFrame(pd.date_range(start = s, end = e).difference(data[date_col]), columns = [date_col])
        missing_dates[aggregator] = item
        for col in fill_cols:
            if fill_val_dict is not None:
                missing_dates[col] = fill_val_dict[col]
            else:
                missing_dates[col] = fill_value
        data = data.append(missing_dates)
        new_df = new_df.append(data)
    return new_df.sort_values(date_col)

In [52]:
def find_fill_missing_dates2(df, aggregator, date_col, fill_cols, copy_val_cols= None, start= None, end= None, fill_value= np.nan, fill_val_dict= None, donor_cols= None):
    '''
    Identifies missing dates, fills missing dates with NaN
    ----
    inputs 
    df: Pandas DataFrame, 
    aggregator: aggregation column (string)
    date_col: primary date-time column (string)
    fill_cols: columns to include (list)
    start (optional): start-date for zero-filling (date-time object)
    end (optional): end-date for zero-filling (date-time object)
    fill_value: default value to place in missing rows + cols (string, numeric, bool)
    fill_val_dict: dictionary of column:fill_value for missing rows (dictionary)

    ----
    outputs: Pandas DataFrame
    '''
    new_df = pd.DataFrame()
    for item in tqdm(df[aggregator].unique().tolist()):
        data = df[df[aggregator] == item]
        s = data[date_col].min()
        e = data[date_col].max()
        if start is not None:
            s = start
        if end is not None:
            e = end
        missing_dates = pd.DataFrame(pd.date_range(start = s, end = e).difference(data[date_col]), columns = [date_col])
        missing_dates[aggregator] = item
        for col in fill_cols:
            if fill_val_dict is not None:
                missing_dates[col] = fill_val_dict[col]
            elif col in donor_cols:
                try:
                    missing_dates[col] = data[col].value_counts().idxmax()
                except: 
                    missing_dates[col] = fill_value
            else:
                missing_dates[col] = fill_value
        data = data.append(missing_dates)
        new_df = new_df.append(data)
    return new_df.sort_values(date_col)

In [66]:
df = pd.read_csv('data/Mars_dataset.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42928 entries, 0 to 42927
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   id                  42928 non-null  object             
 1   Census Regions      42928 non-null  object             
 2   All Markets         42928 non-null  object             
 3   SUB_CATEGORY        42928 non-null  object             
 4   MANUFACTURER        42928 non-null  object             
 5   BRAND               29568 non-null  object             
 6   CONSUMPTION         36206 non-null  object             
 7   SEASONAL PACKAGING  24568 non-null  object             
 8   PRODUCT FORM        24568 non-null  object             
 9   All Products        42928 non-null  object             
 10  target              42928 non-null  float64            
 11  TDP                 42928 non-null  float64            
 12  ACV                 42928 non-nu

In [68]:
df.head()

,id,Census Regions,All Markets,SUB_CATEGORY,MANUFACTURER,BRAND,CONSUMPTION,SEASONAL PACKAGING,PRODUCT FORM,All Products,target,TDP,ACV,Date,WeekOfYr,SOM_Ind,EOM_Ind,year,month,dow,Cluster
0,Midwest Region xAOCGUMHERSHEY CORPBUBBLE YUM B...,Midwest Region xAOC,Midwest Region xAOC,GUM,HERSHEY CORP,BUBBLE YUM BRAND,FC,NON SEASONAL PKG,CHUNK,GUM-HERSHEY CORP-BUBBLE YUM BRAND-FC-NON SEASO...,76.191924,0.172,0.172,2017-08-12 00:00:00+00:00,32,0.0,0.0,2017,8,5,One
1,Midwest Region xAOCGUMHERSHEY CORPBUBBLE YUM B...,Midwest Region xAOC,Midwest Region xAOC,GUM,HERSHEY CORP,BUBBLE YUM BRAND,FC,NaN,NaN,GUM-HERSHEY CORP-BUBBLE YUM BRAND-FC,76.191924,0.172,0.172,2017-08-12 00:00:00+00:00,32,0.0,0.0,2017,8,5,One
2,Midwest Region xAOCGUMHERSHEY CORPBUBBLE YUM B...,Midwest Region xAOC,Midwest Region xAOC,GUM,HERSHEY CORP,BUBBLE YUM BRAND,IC,NON SEASONAL PKG,CHUNK,GUM-HERSHEY CORP-BUBBLE YUM BRAND-IC-NON SEASO...,46737.033123,73.857,41.603,2017-08-12 00:00:00+00:00,32,0.0,0.0,2017,8,5,One
3,Midwest Region xAOCGUMHERSHEY CORPBUBBLE YUM B...,Midwest Region xAOC,Midwest Region xAOC,GUM,HERSHEY CORP,BUBBLE YUM BRAND,IC,NaN,NaN,GUM-HERSHEY CORP-BUBBLE YUM BRAND-IC,46737.033123,73.857,41.603,2017-08-12 00:00:00+00:00,32,0.0,0.0,2017,8,5,One
4,Midwest Region xAOCGUMHERSHEY CORPBUBBLE YUM B...,Midwest Region xAOC,Midwest Region xAOC,GUM,HERSHEY CORP,BUBBLE YUM BRAND,NaN,NaN,NaN,GUM-HERSHEY CORP-BUBBLE YUM BRAND,46814.031586,74.028,41.603,2017-08-12 00:00:00+00:00,32,0.0,0.0,2017,8,5,One


In [69]:
df['id'].nunique()

1104

In [72]:
df['Cluster'].nunique()

1

In [70]:
empty_cols = ['Census Regions', 'All Markets', 'SUB_CATEGORY', 'MANUFACTURER',
       'BRAND', 'CONSUMPTION', 'SEASONAL PACKAGING', 'PRODUCT FORM',
       'All Products', 'target', 'TDP', 'ACV', 'Date', 'WeekOfYr', 'SOM_Ind',
       'EOM_Ind', 'year', 'month', 'dow', 'Cluster']

In [71]:
borrow_cols = ['Census Regions', 'All Markets', 'SUB_CATEGORY', 'MANUFACTURER',
       'BRAND', 'CONSUMPTION', 'SEASONAL PACKAGING', 'PRODUCT FORM',
       'All Products', 'Cluster']

In [14]:
df_filled = find_fill_missing_dates(df= df, aggregator= 'id', date_col= 'Date', fill_cols= empty_cols, start= None, end= None, \
                                    fill_value= np.nan, fill_val_dict= None)

100%|██████████| 1104/1104 [13:35<00:00,  1.35it/s]


In [15]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195472 entries, 0 to 26
Data columns (total 21 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1195472 non-null  object 
 1   Census Regions      42928 non-null    object 
 2   All Markets         42928 non-null    object 
 3   SUB_CATEGORY        42928 non-null    object 
 4   MANUFACTURER        42928 non-null    object 
 5   BRAND               29568 non-null    object 
 6   CONSUMPTION         36206 non-null    object 
 7   SEASONAL PACKAGING  24568 non-null    object 
 8   PRODUCT FORM        24568 non-null    object 
 9   All Products        42928 non-null    object 
 10  target              42928 non-null    float64
 11  TDP                 42928 non-null    float64
 12  ACV                 42928 non-null    float64
 13  Date                42928 non-null    object 
 14  WeekOfYr            42928 non-null    float64
 15  SOM_Ind             

In [53]:
df_filled2 = find_fill_missing_dates2(df= df, aggregator= 'id', date_col= 'Date', fill_cols= empty_cols,  donor_cols= borrow_cols)

100%|██████████| 1104/1104 [12:46<00:00,  1.44it/s]


In [54]:
df_filled2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195472 entries, 0 to 26
Data columns (total 21 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1195472 non-null  object 
 1   Census Regions      1195472 non-null  object 
 2   All Markets         1195472 non-null  object 
 3   SUB_CATEGORY        1195472 non-null  object 
 4   MANUFACTURER        1195472 non-null  object 
 5   BRAND               826427 non-null   object 
 6   CONSUMPTION         1009021 non-null  object 
 7   SEASONAL PACKAGING  686987 non-null   object 
 8   PRODUCT FORM        686987 non-null   object 
 9   All Products        1195472 non-null  object 
 10  target              42928 non-null    float64
 11  TDP                 42928 non-null    float64
 12  ACV                 42928 non-null    float64
 13  Date                42928 non-null    object 
 14  WeekOfYr            42928 non-null    float64
 15  SOM_Ind             

In [55]:
df_filled2.head()

,id,Census Regions,All Markets,SUB_CATEGORY,MANUFACTURER,BRAND,CONSUMPTION,SEASONAL PACKAGING,PRODUCT FORM,All Products,target,TDP,ACV,Date,WeekOfYr,SOM_Ind,EOM_Ind,year,month,dow,Cluster
0,Midwest Region xAOCGUMHERSHEY CORPBUBBLE YUM B...,Midwest Region xAOC,Midwest Region xAOC,GUM,HERSHEY CORP,BUBBLE YUM BRAND,FC,NON SEASONAL PKG,CHUNK,GUM-HERSHEY CORP-BUBBLE YUM BRAND-FC-NON SEASO...,7.619192e+01,0.172,0.172,2017-08-12 00:00:00+00:00,32.0,0.0,0.0,2017.0,8.0,5.0,One
527,Total US FoodGUMHERSHEY CORPHERSHEY AO BRANDSn...,Total US Food,Total US Food,GUM,HERSHEY CORP,HERSHEY AO BRANDS,NaN,NaN,NaN,GUM-HERSHEY CORP-HERSHEY AO BRANDS,6.600000e-02,0.001,0.001,2017-08-12 00:00:00+00:00,32.0,0.0,0.0,2017.0,8.0,5.0,One
528,Total US FoodGUMHERSHEY CORPICE BREAKERS BRAND...,Total US Food,Total US Food,GUM,HERSHEY CORP,ICE BREAKERS BRAND,IC,NON SEASONAL PKG,SOFT CHEWS,GUM-HERSHEY CORP-ICE BREAKERS BRAND-IC-NON SEA...,2.474256e+05,297.012,84.841,2017-08-12 00:00:00+00:00,32.0,0.0,0.0,2017.0,8.0,5.0,One
83,Midwest Region xAOCGUMMONDELEZ INTERNATIONAL I...,Midwest Region xAOC,Midwest Region xAOC,GUM,MONDELEZ INTERNATIONAL INC,BUBBLICIOUS BRAND,FC,NaN,NaN,GUM-MONDELEZ INTERNATIONAL INC-BUBBLICIOUS BRA...,1.715552e+03,0.587,0.584,2017-08-12 00:00:00+00:00,32.0,0.0,0.0,2017.0,8.0,5.0,One
840,Total US xAOC + ConvGUMMONDELEZ INTERNATIONAL ...,Total US xAOC + Conv,Total US xAOC + Conv,GUM,MONDELEZ INTERNATIONAL INC,NaN,IC,NaN,NaN,GUM-MONDELEZ INTERNATIONAL INC-IC,1.806446e+06,1275.602,92.850,2017-08-12 00:00:00+00:00,32.0,0.0,0.0,2017.0,8.0,5.0,One


In [62]:
df['Cluster'].unique()

array(['One'], dtype=object)

In [61]:
df_filled2['Cluster'].unique()

array(['One'], dtype=object)

In [57]:
df_filled.to_csv('data/Mars_filled_nans.csv', index= None)
df_filled2.to_csv('data/Mars_filled_nans_and_donor_vals.csv', index= None)

## Seems like some of the cluster data was missing, here is the full dataset

In [76]:
full_df = pd.read_csv('data/gum-train1.csv_6011d972ac345be28877c0c9 (1).csv')
full_df['Date'] = pd.to_datetime(full_df['Date'])

In [77]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59206 entries, 0 to 59205
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  59206 non-null  object        
 1   Census Regions      59206 non-null  object        
 2   All Markets         59206 non-null  object        
 3   SUB_CATEGORY        59206 non-null  object        
 4   MANUFACTURER        59206 non-null  object        
 5   BRAND               42087 non-null  object        
 6   CONSUMPTION         50590 non-null  object        
 7   SEASONAL PACKAGING  34931 non-null  object        
 8   PRODUCT FORM        34931 non-null  object        
 9   All Products        59206 non-null  object        
 10  target              59206 non-null  float64       
 11  TDP                 59206 non-null  float64       
 12  ACV                 59206 non-null  float64       
 13  Date                59206 non-null  datetime64

In [78]:
full_df.columns

Index(['id', 'Census Regions', 'All Markets', 'SUB_CATEGORY', 'MANUFACTURER',
       'BRAND', 'CONSUMPTION', 'SEASONAL PACKAGING', 'PRODUCT FORM',
       'All Products', 'target', 'TDP', 'ACV', 'Date'],
      dtype='object')

In [80]:
full_empty = ['Census Regions', 'All Markets', 'SUB_CATEGORY', 'MANUFACTURER',\
              'BRAND', 'CONSUMPTION', 'SEASONAL PACKAGING', 'PRODUCT FORM',\
              'All Products', 'target', 'TDP', 'ACV']
full_borrow = ['Census Regions', 'All Markets', 'SUB_CATEGORY', 'MANUFACTURER',\
            'BRAND', 'CONSUMPTION', 'SEASONAL PACKAGING', 'PRODUCT FORM','All Products']

In [81]:
full_df_filled = find_fill_missing_dates2(df= full_df, aggregator= 'id', date_col= 'Date', fill_cols= full_empty,  donor_cols= full_borrow)

100%|██████████| 1864/1864 [08:42<00:00,  3.56it/s]


In [84]:
full_df_filled['Date'] = pd.to_datetime(full_df_filled['Date'])

In [85]:
full_df_filled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1754888 entries, 0 to 34957
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   id                  object        
 1   Census Regions      object        
 2   All Markets         object        
 3   SUB_CATEGORY        object        
 4   MANUFACTURER        object        
 5   BRAND               object        
 6   CONSUMPTION         object        
 7   SEASONAL PACKAGING  object        
 8   PRODUCT FORM        object        
 9   All Products        object        
 10  target              float64       
 11  TDP                 float64       
 12  ACV                 float64       
 13  Date                datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(10)
memory usage: 200.8+ MB


In [107]:
# engineer some columns
full_df_filled['WeekofYr'] = full_df_filled['Date'].apply(lambda x: x.week)
full_df_filled['year'] = full_df_filled['Date'].apply(lambda x: x.year)
full_df_filled['month'] = full_df_filled['Date'].apply(lambda x: x.month)
full_df_filled['dow'] = full_df_filled['Date'].apply(lambda x: x.dayofweek)

In [121]:
def check_special_date(date, check= 'last'):
    import datetime
    import calendar

    if check == 'last':
        special_date = calendar.monthrange(date.year, date.month)[1]
        if date == datetime.date(date.year, date.month, special_date):
            return True
    elif check == 'first':
        special_date = date.replace(day=1)
        if date.day == special_date.day:
            return True
    else:
        print('The value for "check" must be either "first" or "last".')
        return False

In [125]:
full_df_filled['EOM_ind'] = full_df_filled['Date'].apply(lambda x: check_special_date(x, 'last'))
full_df_filled['FOM_ind'] = full_df_filled['Date'].apply(lambda x: check_special_date(x, 'first'))
full_df_filled['EOM_ind'].fillna(False, inplace= True)
full_df_filled['FOM_ind'].fillna(False, inplace= True)

In [128]:
full_df_filled.to_csv('data/Mars_all_data_filled_nans_and_donor_vals_with_FE.csv', index= None)